# Neural Network

A neural network is created which can be used for training and testing on trips data.



In [1]:
!pip install tensorflow

In [10]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [11]:
path_output=os.path.join(os.getcwd(), "..", "data", "output")

In [12]:
features = pd.read_csv(os.path.join(path_output, "Features.csv"))
features = features.drop("Unnamed: 0", axis=1)

### Take Params from csv

In [13]:
params = pd.read_csv(os.path.join(os.getcwd(), "..", "data", "input", "params.csv")).drop("0", axis=1)
_test_size = params[params["param"]=="test_size"]["value"].values[0]
_random_state = int(params[params["param"]=="random_state"]["value"].values[0])
_epochs = int(params[params["param"]=="epochs"]["value"].values[0])
_validation_size = params[params["param"]=="validation_size"]["value"].values[0]

### Neural Network

In [14]:
def train_neural_network(X_train_scaled, y_train, on, hex_size):
    """
    Train Neural Network Model

    Train and save a Neural Network model.
    The network has the following properties:
        - three hidden layer
        - 50 epochs
        - activation function is relu
        - dimension of input and hidden layer is 36
        - dimension of output layer is 1
        - dropout is not used
    Then evaluate the error metrics by another method.

    Args:
        X_train_scaled (DataFrame):   Scaled X input of train set (matrix)
        y_train (Series):             y output to train on (vector)
    Returns:
        nn_regression_sets (array): true y values and predicted y values for train and validation set
    """
    # create a validation set which is 20% of the whole dataset. Therefore use formula to receive ca. 0.2857.
    X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y_train, random_state=_random_state, test_size=_validation_size)
    neural_network = keras.Sequential(
        [layers.Dense(36, activation="relu", input_shape=[X_train_scaled.shape[1]], kernel_initializer="random_normal"),
         # layers.Dropout(0.2),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         # layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         # layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         # layers.Dense(36, activation="softmax"),
         # layers.Dense(36, activation="softmax"),
         # layers.Dropout(0.2),
         layers.Dense(1)])
    optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
    neural_network.compile(loss="mse",
                           optimizer=optimizer,
                           metrics=["mae", "mse"])
    epochs = _epochs
    # create a validation set which is 20% of the whole dataset. Therefore use formula to receive ca. 0.2857.
    history = neural_network.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))
    neural_network.save(os.path.join(path_output, "models", "NN_Regression_Model_"+on+"_"+hex_size))
    y_prediction_train = neural_network.predict(X_train)
    y_prediction_val = neural_network.predict(X_val)
    
    r2_train = r2_score(y_train, y_prediction_train)
    print("R2 (Train):",r2_train)
    r2_val = r2_score(y_val, y_prediction_val)
    print("R2 (Validation):",r2_val)
    
    plot_train_loss(history, on=on, hex_size=hex_size)
    nn_regression_sets = [y_train, y_val, y_prediction_train, y_prediction_val]
    return nn_regression_sets

### Loss visualization by epoch

In [15]:
def plot_train_loss(history, on, hex_size):
    """
    Plot the train and validation loss of Neural Network.

    Args:
        history (Object): History of loss during training of neural network
        on (str): time resolution to train on
    Returns:
        No return
    """
    print("Plot training and visualization loss...")
    # Plotting the training and validation loss
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    epochs = range(1, len(loss) + 1)
    fig, ax = plt.subplots(figsize=(16, 8), dpi=300)
    ax.plot(epochs, loss, "bo", label="Training loss")
    ax.plot(epochs, val_loss, "b", label="Validation loss")
    ax.set_title("Training and validation loss "+on+"_"+hex_size, fontsize=18)
    ax.set_xlabel("Epochs", fontsize=16)
    ax.set_ylabel("Loss", fontsize=16)
    plt.legend()
    #plt.show()
    fig.savefig(os.path.join(path_output, "NN_error_per_epoch_"+on+"_"+hex_size+".png"))
    plt.close(fig)

### Run NN

In [16]:
def train_NN(on="24_demand", hex_size="hexa_big"):
    """
    Split the data in train and test set by 0.3 test set. 
    Then Scale the data and do a PCA. 
    Last train the NN on the chosen time resolution
    
    Args:
        on (str): time resolution to train on
        
    Returns:
        No return
    """
    print("Time Resolution is", on)
    #print("Split Data with random state", _random_state, "and test size", str(_test_size)+"...")
    # features_X = features.drop(["24_sum", "6_sum", "2_sum", "1_sum"], axis=1)
    features_X = features.drop(["24_demand", "24_demand_hex_big", "24_demand_hex_small", "24_agg_time",
                                "6_demand", "6_demand_hex_big", "6_demand_hex_small", "6_agg_time",
                                "2_demand", "2_demand_hex_big", "2_demand_hex_small", "2_agg_time",
                                "1_demand", "1_demand_hex_big", "1_demand_hex_small", "1_agg_time",
                                "24_available_hex_big"], axis=1)
    features_y = features[on]
    
    # Spatial Resolution
    print("Spatial Resolution is", hex_size)
    if hex_size=="hexa_small":
        features_X = features_X.drop("hexa_big", axis=1)
    else:
        features_X = features_X.drop("hexa_small", axis=1)
    
    #Split
    X_train, X_test, y_train, y_test = train_test_split(features_X, features_y, random_state=_random_state, test_size=_test_size)

    #print("Scale", hex_size, "Data with Standard Scaler...")
    with open(os.path.join(path_output, "models", "Standard_Scaler_"+hex_size+".pkl"), "rb") as f:
        standard_scaler = pickle.load(f)
    X_train_scaled = standard_scaler.transform(X_train)

    #print("Do PCA on", hex_size, "Data...")
    with open(os.path.join(path_output, "models", "PCA_"+hex_size+".pkl"), "rb") as f:
        pca = pickle.load(f)
    X_train_transformed = pca.transform(X_train_scaled)

    print("Train", "NN_Regression_Model_"+on+"_"+hex_size, "...")
    nn_regression_sets = train_neural_network(X_train_transformed, y_train.to_numpy(), on=on, hex_size=hex_size)

In [17]:
#Train the NN for each time resolution.
# hex_size = ["hexa_small", "hexa_big"]
hex_size = ["hexa_big", "hexa_small"]
time_resolutions = ["24_demand", "6_demand", "2_demand", "1_demand"]

# """
for time in time_resolutions:
    for size in hex_size:
        train_NN(on=time, hex_size=size)
        print()
print("Done")
# """

train_NN(on="24_available_hex_big", hex_size="hexa_big")

Time Resolution is 24_demand
Spatial Resolution is hexa_big
Train NN_Regression_Model_24_demand_hexa_big ...
Epoch 1/20
8867/8867 [==============================] - 17s 2ms/step - loss: 1073795.6250 - mae: 644.5992 - mse: 1073795.6250 - val_loss: 484372.5000 - val_mae: 455.5758 - val_mse: 484372.5000
Epoch 2/20
8867/8867 [==============================] - 14s 2ms/step - loss: 390321.6562 - mae: 371.1285 - mse: 390321.6562 - val_loss: 262740.6250 - val_mae: 344.1538 - val_mse: 262740.6250
Epoch 3/20
8867/8867 [==============================] - 14s 2ms/step - loss: 190393.6250 - mae: 288.5427 - mse: 190393.6250 - val_loss: 137350.2812 - val_mae: 247.5357 - val_mse: 137350.2812
Epoch 4/20
8867/8867 [==============================] - 15s 2ms/step - loss: 120800.2500 - mae: 230.2508 - mse: 120800.2500 - val_loss: 117706.2109 - val_mae: 235.2872 - val_mse: 117706.2109
Epoch 5/20
8867/8867 [==============================] - 15s 2ms/step - loss: 111292.0391 - mae: 187.6297 - mse: 111292.0391 -


Time Resolution is 6_demand
Spatial Resolution is hexa_big
Train NN_Regression_Model_6_demand_hexa_big ...
Epoch 1/20
8867/8867 [==============================] - 14s 2ms/step - loss: 333816.9688 - mae: 331.0051 - mse: 333816.9688 - val_loss: 132205.4375 - val_mae: 232.8709 - val_mse: 132205.4375
Epoch 2/20
8867/8867 [==============================] - 13s 1ms/step - loss: 158365.5781 - mae: 221.2687 - mse: 158365.5781 - val_loss: 95368.3281 - val_mae: 197.4160 - val_mse: 95368.3281
Epoch 3/20
8867/8867 [==============================] - 13s 1ms/step - loss: 96325.7266 - mae: 190.6429 - mse: 96325.7266 - val_loss: 77950.1328 - val_mae: 183.3965 - val_mse: 77950.1328
Epoch 4/20
8867/8867 [==============================] - 13s 1ms/step - loss: 77321.7344 - mae: 171.1252 - mse: 77321.7344 - val_loss: 68069.6875 - val_mae: 158.0697 - val_mse: 68069.6875
Epoch 5/20
8867/8867 [==============================] - 16s 2ms/step - loss: 65485.2031 - mae: 156.0303 - mse: 65485.2031 - val_loss: 5480


Time Resolution is 2_demand
Spatial Resolution is hexa_big
Train NN_Regression_Model_2_demand_hexa_big ...
Epoch 1/20
8867/8867 [==============================] - 15s 2ms/step - loss: 118683.3359 - mae: 211.2008 - mse: 118683.3359 - val_loss: 79832.5000 - val_mae: 181.1735 - val_mse: 79832.5000
Epoch 2/20
8867/8867 [==============================] - 16s 2ms/step - loss: 82514.4062 - mae: 153.7110 - mse: 82514.4062 - val_loss: 51590.6484 - val_mae: 142.5133 - val_mse: 51590.6484
Epoch 3/20
8867/8867 [==============================] - 15s 2ms/step - loss: 53466.7461 - mae: 127.8971 - mse: 53466.7461 - val_loss: 33845.6875 - val_mae: 125.9688 - val_mse: 33845.6875
Epoch 4/20
8867/8867 [==============================] - 16s 2ms/step - loss: 29726.3281 - mae: 112.3401 - mse: 29726.3281 - val_loss: 25209.7070 - val_mae: 104.1068 - val_mse: 25209.7070
Epoch 5/20
8867/8867 [==============================] - 15s 2ms/step - loss: 24718.7676 - mae: 102.0371 - mse: 24718.7676 - val_loss: 21392.68

Epoch 1/20
8867/8867 [==============================] - 13s 2ms/step - loss: 57624.4648 - mae: 157.6337 - mse: 57624.4648 - val_loss: 38913.2188 - val_mae: 128.7477 - val_mse: 38913.2188
Epoch 2/20
8867/8867 [==============================] - 13s 1ms/step - loss: 31624.4531 - mae: 119.7694 - mse: 31624.4531 - val_loss: 31299.2090 - val_mae: 124.4142 - val_mse: 31299.2090
Epoch 3/20
8867/8867 [==============================] - 13s 1ms/step - loss: 24014.2969 - mae: 101.0903 - mse: 24014.2969 - val_loss: 21175.3496 - val_mae: 93.9656 - val_mse: 21175.3496
Epoch 4/20
8867/8867 [==============================] - 14s 2ms/step - loss: 20375.0000 - mae: 88.6168 - mse: 20375.0000 - val_loss: 17281.8770 - val_mae: 85.1457 - val_mse: 17281.8770
Epoch 5/20
8867/8867 [==============================] - 13s 1ms/step - loss: 16613.0723 - mae: 79.9668 - mse: 16613.0723 - val_loss: 15757.3838 - val_mae: 77.0059 - val_mse: 15757.3838
Epoch 6/20
8867/8867 [==============================] - 13s 2ms/step -

Epoch 2/20
8867/8867 [==============================] - 15s 2ms/step - loss: 19511.5527 - mae: 114.8405 - mse: 19511.5527 - val_loss: 17940.1816 - val_mae: 113.6077 - val_mse: 17940.1816
Epoch 3/20
8867/8867 [==============================] - 15s 2ms/step - loss: 18365.3535 - mae: 110.9280 - mse: 18365.3535 - val_loss: 16889.6504 - val_mae: 108.6115 - val_mse: 16889.6504
Epoch 4/20
8867/8867 [==============================] - 14s 2ms/step - loss: 16661.5977 - mae: 106.5646 - mse: 16661.5977 - val_loss: 16443.7090 - val_mae: 105.9245 - val_mse: 16443.7090
Epoch 5/20
8867/8867 [==============================] - 15s 2ms/step - loss: 16083.3516 - mae: 102.4462 - mse: 16083.3516 - val_loss: 15230.0137 - val_mae: 100.6009 - val_mse: 15230.0137
Epoch 6/20
8867/8867 [==============================] - 13s 1ms/step - loss: 14778.3066 - mae: 97.1085 - mse: 14778.3066 - val_loss: 14110.5645 - val_mae: 94.9629 - val_mse: 14110.5645
Epoch 7/20
8867/8867 [==============================] - 14s 2ms/ste